In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from lib_hcy_week5 import myfunctions as mf
from scipy.stats import t, norm

In [2]:
df = pd.read_csv("DailyPrices.csv")
all_ret = mf.return_calculate(df)
all_ret.head()

/Users/yanghaochen/opt/anaconda3/lib/python3.9/site-packages/lib_hcy_week5/myfunctions.py:243: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  out[vars[i]] = p2[:,i]
/Users/yanghaochen/opt/anaconda3/lib/python3.9/site-packages/lib_hcy_week5/myfunctions.py:243: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  out[vars[i]] = p2[:,i]


,Date,SPY,AAPL,MSFT,AMZN,NVDA,GOOGL,TSLA,GOOG,BRK-B,...,CI,ETN,SLB,PGR,SCHW,LRCX,ZTS,C,BSX,AMT
1,2022-09-02,-0.010544,-0.013611,-0.016667,-0.002425,-0.020808,-0.017223,-0.025076,-0.016915,-0.016854,...,-0.001180,-0.010593,0.033107,-0.010428,-0.019242,-0.004236,-0.015244,0.001846,-0.012198,-0.026355
2,2022-09-06,-0.003773,-0.008215,-0.010974,-0.010980,-0.013336,-0.009643,0.015581,-0.011042,-0.003890,...,-0.004641,0.008449,-0.014118,0.000572,0.001848,-0.008019,-0.000892,-0.012695,-0.002717,0.013275
3,2022-09-07,0.017965,0.009254,0.019111,0.026723,0.018795,0.024717,0.033817,0.027912,0.016089,...,0.016652,0.020295,-0.008030,0.038537,0.018731,0.012279,0.022698,0.008503,0.026994,0.020930
4,2022-09-08,0.006536,-0.009618,0.001666,0.002626,0.020126,-0.009776,0.019598,-0.009595,0.008184,...,0.002448,0.013945,0.029951,0.015880,0.019083,0.016574,-0.011908,0.026116,0.029901,0.008362
5,2022-09-09,0.015535,0.018840,0.022977,0.026575,0.028377,0.020945,0.036023,0.021568,0.008576,...,0.007327,0.017244,0.038774,-0.004179,0.018863,0.026460,0.036721,0.015431,0.005385,-0.000306


In [3]:
all_ret.drop('Date', axis=1, inplace=True)

In [8]:
ret_data = all_ret
w_lambda = 0.97
weight = np.zeros(ret_data.shape[0]) # create a vector with num of rows corresponding to the periods of time, so we can attach weight to each time point
for i in range(len(ret_data)):
    weight[len(ret_data)-1-i]  = (1-w_lambda)*w_lambda**i # weight for each time point will be decided by lambda set
weight = weight/sum(weight) # assure that sum_weight == 1
adj_ret_means = ret_data - ret_data.mean()
print("adj_ret_means:")
print(adj_ret_means.iloc[0,:])
print("adj_ret_means.values")
print(adj_ret_means.values[0])

adj_ret_means:
SPY    -0.010992
AAPL   -0.014177
MSFT   -0.017632
AMZN   -0.002763
NVDA   -0.025527
          ...   
LRCX   -0.006037
ZTS    -0.015836
C       0.002191
BSX    -0.013291
AMT    -0.024982
Name: 1, Length: 101, dtype: float64
adj_ret_means.values
[-0.01099246 -0.01417705 -0.01763241 -0.00276286 -0.02552684 -0.01809861
 -0.02533128 -0.01779749 -0.01783563 -0.03326605 -0.01463848  0.01728739
 -0.02760476 -0.00813236 -0.01577608 -0.01256118 -0.01821639 -0.00919124
  0.01436841 -0.00539698  0.01433484 -0.01151881 -0.01628668 -0.01934489
 -0.01287417 -0.007986   -0.01182796 -0.01361558 -0.01630324 -0.00032733
 -0.00799747 -0.01689314 -0.00073615 -0.01287004 -0.03192812 -0.01884637
 -0.00636906 -0.00986486 -0.02224191 -0.01937903 -0.01601666 -0.02650658
 -0.00737637 -0.0108095  -0.02042019 -0.01914054 -0.01771958 -0.01329777
  0.02458022 -0.01375407 -0.0116879  -0.0106049  -0.01263279 -0.01514899
 -0.00932205 -0.02028006 -0.00653439 -0.00178473 -0.01157429 -0.01271015
 -0.010754

In [14]:
weights = pd.DataFrame(np.diag(weight), index=adj_ret_means.index)
weights.columns = range(1, len(weights.columns) + 1)
print(weights)

adj = pd.DataFrame(np.diag(weight), index=adj_ret_means.index, )

expo_w_cov = adj_ret_means.T @ np.diag(weight) @ adj_ret_means
print(expo_w_cov)
# np.diag(weight) is not a

         1        2        3         4         5    6    7    8    9    10   \
1    0.00001  0.00000  0.00000  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0   
2    0.00000  0.00001  0.00000  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0   
3    0.00000  0.00000  0.00001  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0   
4    0.00000  0.00000  0.00000  0.000011  0.000000  0.0  0.0  0.0  0.0  0.0   
5    0.00000  0.00000  0.00000  0.000000  0.000011  0.0  0.0  0.0  0.0  0.0   
..       ...      ...      ...       ...       ...  ...  ...  ...  ...  ...   
261  0.00000  0.00000  0.00000  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0   
262  0.00000  0.00000  0.00000  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0   
263  0.00000  0.00000  0.00000  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0   
264  0.00000  0.00000  0.00000  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0   
265  0.00000  0.00000  0.00000  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0   

     ...  256  257  258  259  260       261       2

ValueError: matrices are not aligned